In [1]:
from pyspark import SparkContext
from algorithms.content_fede_funzionante import calculate_content_matrix
from algorithms.user_user_fede import calculate_user_collaborative_matrix
import csv

sc = SparkContext.getOrCreate()

train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2]))).cache()
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]))).cache()
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(',')).cache()

In [2]:
content_fede = calculate_content_matrix(train_clean_data, icm_clean_data, test_clean_data)


In [3]:
collaborative_user = calculate_user_collaborative_matrix(train_clean_data, test_clean_data)

In [4]:
from operator import add
from itertools import groupby
from operator import itemgetter

def add_weights(user_rates):
    return user_rates[0],[(item, rating * 0.5)for item,rating in user_rates[1]]



def union_ratings(user_rates):
    ratings = list()
    for rates in user_rates[1]:
        ratings += rates
    return user_rates[0], ratings


def sum_weights(user_rates):
    ratings = user_rates[1]
    ratings = sorted(ratings, key=lambda x: x[0])
    ratings = [(x,list(map(itemgetter(1),y))) for x,y in groupby(ratings, itemgetter(0))]
    result = list(map(lambda x: (x[0], sum(x[1])),ratings))
    return user_rates[0], result



In [5]:
#rdd2a = collaborative_user.map(add_weights)


In [6]:

rdd3 = content_fede.union(collaborative_user).reduceByKey(add)

rdd4 = rdd3.map(sum_weights)


In [7]:
ciao = rdd4.take(1)

In [8]:
ciao[:5]

[(796,
  [(-9.0951367036841, 7114),
   (-7.677928871477259, 20172),
   (-7.4, 21446),
   (-7.158557229129811, 9840),
   (-6.463235294117647, 40734),
   (-6.341523341523342, 41407),
   (-6.325074925074926, 30616),
   (-5.815546772068511, 15180),
   (-5.786884563917976, 32117),
   (-5.774625735912531, 16356),
   (-5.677928871477259, 35856),
   (-5.625, 31836),
   (-5.378899835796388, 3800),
   (-5.3478260869565215, 27629),
   (-5.341523341523342, 114838),
   (-5.34020618556701, 22158),
   (-5.336507936507936, 15089),
   (-5.336405529953917, 121656),
   (-5.273932650966064, 27828),
   (-5.122855219435415, 24012),
   (-5.02, 15730),
   (-4.982300884955753, 36237),
   (-4.963694765952143, 23089),
   (-4.904761904761905, 32552),
   (-4.885425685425686, 32743),
   (-4.750487804878048, 11458),
   (-4.742603681387412, 19452),
   (-4.7190082644628095, 2127),
   (-4.689558629776021, 31138),
   (-4.6256530825496345, 28028),
   (-4.616505706490428, 14441),
   (-4.580626137147876, 21381),
   (-4.562